In [ ]:
#default_exp explorer

In [ ]:
#export
from fastai2.vision.all import *
from fastexplorer.representation import *
from starlette.applications import Starlette
from starlette.endpoints import WebSocketEndpoint
from starlette.websockets import WebSocketDisconnect
import uvicorn, asyncio, json, logging
import nest_asyncio
nest_asyncio.apply()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
path = untar_data(URLs.MNIST_TINY)
dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False)

# Fast Explorer

> Here we will setup the server and store the model representation on the fastai `Learn` object..

In [ ]:
#export
#TODO EVENT ENUMS

In [ ]:
#export
class FastExplorer:
    'Wrapper around `Representation` and `ProxyServer`.'
    def __init__(self, learn, host='0.0.0.0', port=8000):
        store_attr(self, 'learn,host,port')
        self.representation = learn.to_representation()
        self.server = Starlette()
        self.endpoint = self.server.websocket_route('/ws')(self.endpoint)
        self.socket = None
        
    def __repr__(self): return f'{self.__class__.__name__} ()'

    async def handle_web_client(self, websocket):
        while True:
            try:
                msg = await websocket.receive_json()
                logging.info(f'Client msg: {msg}')
                await self.dispatch_event(websocket, msg)
            except json.JSONDecodeError: logging.info('Invalid JSON received')
            except WebSocketDisconnect: break
    
    async def dispatch_event(self, websocket, msg):
        typ = msg['type']
        payload = msg['payload']
        
        if typ != 'socket/sendEvent':
            logging.info(f'Invalid type: {typ!r}')
        else:
            event = typ.split('/')[1]
            if not hasattr(self, event):
                logging.info(f'Invalid event: {event!r}')
                await websocket.send_json({'type': 'socket/socketInvalidEvent', 'payload': {'type': typ}})
            else:
                logging.info('Should do something here #TODO...')
    
    async def endpoint(self, websocket):
        # Open socket
        await websocket.accept()
        msg = await websocket.receive_json()
        self.socket = websocket
        logging.info(f'Client connected: {msg}')
        
        # Manage requests
        await websocket.send_json({
                'type': 'socket/socketReceiveData',
                'payload': { 'data': self.representation.to_json()}
            })
        await self.handle_web_client(websocket)
        
        # Close socket
        await close_client(websocket, 'Finalized by server')
        logging.info(f'Client closed: {msg}')    
    
    def serve(self): uvicorn.run(self.server, host=self.host, port=self.port)

In [ ]:
#export
async def close_client(websocket, msg):
    await websocket.send_json({'event': 'close', 'msg': msg})
    await websocket.close()

In [ ]:
#export
@patch
@delegates(FastExplorer.__init__)
def fastexplorer(self:Learner, reload=False, **kwargs):
    if (not hasattr(self, 'explorer')) or reload: self.explorer = FastExplorer(self, **kwargs)
    self.explorer.serve()

In [ ]:
#hide
learn.fastexplorer(True)

INFO:     Started server process [7609]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 50830) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'type': 'CONNECTED', 'payload': {'client': 'web_client'}}
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': 'LOAD_INPUT'}
INFO:     Invalid event: 'sendEvent'
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 153, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/ap

INFO:     ('127.0.0.1', 51398) - "WebSocket /ws" [accepted]
INFO:     Client connected: {'type': 'CONNECTED', 'payload': {'client': 'web_client'}}
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': 'LOAD_INPUT'}
INFO:     Invalid event: 'sendEvent'
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 153, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/middleware/errors.py", li

INFO:     Client connected: {'type': 'CONNECTED', 'payload': {'client': 'web_client'}}
INFO:     Client msg: {'type': 'socket/sendEvent', 'payload': 'LOAD_INPUT'}
INFO:     Invalid event: 'sendEvent'
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 153, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/middleware/errors.py", line 146, in __call__
    await self.app(scope, receive, send)

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 153, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/uvicorn/middleware/proxy_headers.py", line 45, in __call__
    return await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/middleware/errors.py", line 146, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib/python3.8/site-packages/starlette/exceptions.py", line 58, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml2/lib

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_representation.ipynb.
Converted 01_explorer.ipynb.
Converted 10_tutorial.ipynb.
Converted index.ipynb.
